In [ ]:
import pandas as pd

# for 1.4
from sklearn.preprocessing import LabelEncoder

# for 1.5 & 2.5
import matplotlib.pyplot as plt
import seaborn as sns

# for 1.6
import re

# for 1.7
from sklearn.feature_extraction.text import TfidfVectorizer

# for 2.2
from sklearn.model_selection import train_test_split

# for 2.3
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# for 2.4 & 2.5
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# task 1.1
df = pd.read_excel("../data/raw/email_spam.xlsx")
print(df)

# task 1.3
# remove useless columns
columns_to_delete = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"];
df = df.drop(columns=columns_to_delete)
print(df.info())

# task 1.2
# remove same string seen twice in the v2 column
df = df.drop_duplicates(subset=['v2'])
print(df.info())

# task 1.4
# encoding column v1
le = LabelEncoder()
df['target_encoded'] = le.fit_transform(df['v1'])
print(df.head())

# task 1.5
# distribution
plt.figure(figsize=(6, 4))
sns.countplot(x='v1', data=df)
plt.title('Ham vs Spam Distribution')
plt.show()

# task 1.6
# function that filter text
def clean_text(text):
    # lowercase letters
    text = str(text).lower()
    # keep only the letters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # eliminate multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# apply the function created above to filter the v2 column
df['text_clean'] = df['v2'].apply(clean_text)
print(df[['v2', 'text_clean']].head())

# task 1.7
# transform text_clean -> vector
# max 5000 features
tfidf = TfidfVectorizer(max_features=5000)

# learns the vocabulary
X = tfidf.fit_transform(df['text_clean'])
print("Matrix Size (rows, cols (words)):")
print(X.shape)

# task 2.1
# separate features (X) from targer var (y)
y = df['target_encoded']

# task 2.2
# split data into training (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# task 2.3 - prepare different models for training
model_params = {
    'Naive Bayes': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.1, 0.5, 1.0]
        }
    },
    'Support Vector Machine': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    }
}

# loop through models
for model_name, mp in model_params.items():
    print(f"\n--- Processing {model_name} ---")

    # task 2.3: train models with different hyperparameter combinations using GridSearch
    clf = GridSearchCV(mp['model'], mp['params'], cv=3, n_jobs=-1)
    clf.fit(X_train, y_train)

    # task 2.4: evaluate the best model
    best_model = clf.best_estimator_
    print(f"Best Hyperparameters: {clf.best_params_}")

    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # task 2.5: display confusion matrix as a heatmap
    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

# save processed data for check
df.to_excel("../data/processed/email_spam_processed.xlsx", index=False)

ModuleNotFoundError: No module named 'pandas'